In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import os
import numpy as np

In [30]:
from google.colab import files
uploaded = files.upload()

Saving Sample.txt to Sample.txt


In [ ]:
file = open("Sample.txt","r",encoding="utf8")

lines=[]
for i in file:
  lines.append(i)

#converting list to string
data = ''
for i in lines:
  data = ' '.join(lines)

#replacing unnecessary characters with space
data = data.replace('\n','').replace("\r",'').replace('\ufeff','').replace('"','').replace('*','').replace(',','')

#removing unnecessary spaces
data = data.split()
data = ' '.join(data)
print(data[:500])
data[:500]
print('Length: ',len(data))

eadful measures. Grim-visaged war hath smooth'd his wrinkled front; And now instead of York; And now instead of our house In the deep bosom of fearful measures. Grim-visaged war hath smooth'd his wrinkled for sportion us pleasing barded steeds To the souls of fearful adversaries Nor monuments; Our bruised arms hung up for sportive tricks Now is the winter of York; And now instead of the lascivious pleasing of York; And all the ocean buried. Nor monuments; Our brows bound war hath smooth'd his fa
Length:  3377


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

#saving the tokenizer for predict function
pickle.dump(tokenizer,open('token.pkl','wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]
print("Length: ",len(sequence_data))

Length:  582


In [ ]:
vocab_size = len(tokenizer.word_index)+1
print(vocab_size)

109


In [ ]:
sequences = []
for i in range(3,len(sequence_data)):
  words = sequence_data[i-3:i+1]
  sequences.append(words)

print("Length of sequences: ",len(sequences))
sequences = np.array(sequences)
sequences[:10]


Length of sequences:  579


array([[99, 40, 41, 42],
       [40, 41, 42, 15],
       [41, 42, 15, 16],
       [42, 15, 16, 17],
       [15, 16, 17, 18],
       [16, 17, 18, 54],
       [17, 18, 54, 21],
       [18, 54, 21,  6],
       [54, 21,  6, 11],
       [21,  6, 11, 19]])

In [ ]:
x=[]
y=[]
for i in sequences:
  x.append(i[0:3])
  y.append(i[3])

#x data acts as input which is used for prediction.
x = np.array(x)
#y is response data which is predicted based on x
y = np.array(y)

In [ ]:
print("Data: \n",x[:10])
print("Response: \n",y[:10])

Data: 
 [[99 40 41]
 [40 41 42]
 [41 42 15]
 [42 15 16]
 [15 16 17]
 [16 17 18]
 [17 18 54]
 [18 54 21]
 [54 21  6]
 [21  6 11]]
Response: 
 [42 15 16 17 18 54 21  6 11 19]


In [ ]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0.

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size,10,input_length=3))
model.add(LSTM(1000,return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000,activation="relu"))
model.add(Dense(vocab_size,activation="softmax"))

In [ ]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 3, 10)             1090      
                                                                 
 lstm_1 (LSTM)               (None, 3, 1000)           4044000   
                                                                 
 lstm_2 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense_1 (Dense)             (None, 1000)              1001000   
                                                                 
 dense_2 (Dense)             (None, 109)               109109    
                                                                 
Total params: 13159199 (50.20 MB)
Trainable params: 13159199 (50.20 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5",monitor="loss",verbose=1,save_best_only=True)
model.compile(loss="categorical_crossentropy",optimizer=Adam(learning_rate=0.001))
model.fit(x,y,epochs=30,batch_size=64,callbacks=[checkpoint])

Epoch 1/30
10/10 [==============================] - ETA: 0s - loss: 4.6680
Epoch 1: loss improved from inf to 4.66795, saving model to next_words.h5
10/10 [==============================] - 6s 348ms/step - loss: 4.6680
Epoch 2/30


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


10/10 [==============================] - ETA: 0s - loss: 4.4704
Epoch 2: loss improved from 4.66795 to 4.47038, saving model to next_words.h5
10/10 [==============================] - 3s 271ms/step - loss: 4.4704
Epoch 3/30
10/10 [==============================] - ETA: 0s - loss: 4.4886
Epoch 3: loss did not improve from 4.47038
10/10 [==============================] - 2s 245ms/step - loss: 4.4886
Epoch 4/30
10/10 [==============================] - ETA: 0s - loss: 4.3806
Epoch 4: loss improved from 4.47038 to 4.38063, saving model to next_words.h5
10/10 [==============================] - 3s 261ms/step - loss: 4.3806
Epoch 5/30
10/10 [==============================] - ETA: 0s - loss: 4.3433
Epoch 5: loss improved from 4.38063 to 4.34334, saving model to next_words.h5
10/10 [==============================] - 3s 300ms/step - loss: 4.3433
Epoch 6/30
10/10 [==============================] - ETA: 0s - loss: 4.2977
Epoch 6: loss improved from 4.34334 to 4.29773, saving model to next_words.h5
1

In [ ]:
from tensorflow.keras.models import load_model


#load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl','rb'))

def Predict_Next_Words(model,tokenizer,text):
  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predict_word = ""

  for key,value in tokenizer.word_index.items():
    if value == preds:
      predicted_word = key
      break

  print(predicted_word)
  return predicted_word

In [ ]:
while (True):
  text = input("Enter your line: ")

  if text=="0":
    print("Execution Terminated...")
    break
  else:
    try:
      text = text.split(" ")
      text = text[-3:]
      print(text)
      Predict_Next_Words(model,tokenizer,text)
    except Exception as e:
      print("Error Occured: ",e)
      continue

Enter your line: bound war hath smooth'd his fair
["smooth'd", 'his', 'fair']
1/1 [==============================] - 0s 22ms/step
proportive
Enter your line: fright the lascivious
['fright', 'the', 'lascivious']
1/1 [==============================] - 0s 21ms/step
summer
